In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem, rdMolTransforms
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.molSize = (400,400)
IPythonConsole.drawOptions.addAtomIndices = True

In [ ]:
mol = Chem.MolFromMolFile("./889.sdf", removeHs=False)

In [ ]:
mol

In [ ]:
rdMolTransforms.GetDihedralDeg(mol.GetConformer(), 20, 19, 18, 6)

Create the explicit simulation

In [ ]:
from Simulation.Simulator import Simulator
from ForceField.Forcefield import OpenFF_forcefield
from openmm import LangevinMiddleIntegrator, MonteCarloBarostat
import mdtraj

setup the simulation

In [ ]:
smiles = Chem.MolToSmiles(mol)
explicit_simulation = Simulator(work_dir="/tmp/explicit_simulation",
                                name="explicit_simulation",
                                save_name="889",
                                pdb_id=f"{smiles}_in_O",
                                rdkit_mol=mol)
explicit_simulation.forcefield = OpenFF_forcefield(f"{smiles}_in_O",rdkit_mol=mol,cache="/tmp/explicit_simulation")
explicit_simulation.integrator = LangevinMiddleIntegrator(300, 1.0, 2.0e-3)
explicit_simulation.barostat = MonteCarloBarostat(1.0, 300, 25)

restrain the molecule

In [ ]:
num_particles = explicit_simulation._system.getNumParticles()
num_atoms = mol.GetNumAtoms()

# Molecule atoms are last in the system
atomsToRestrain = [i + num_particles - num_atoms for i in range(num_atoms)]

# OpenMM restrains all atoms with mass 0
for i in atomsToRestrain:
    explicit_simulation._system.setParticleMass(i, 0.0)

# Create simulation to apply restraints
explicit_simulation.create_simulation()
explicit_simulation.platform = "GPU"

run the simulation

In [ ]:
# Run Simulation, 200 ps
explicit_simulation.run_simulation(100000,20)

analyze trajectory

In [ ]:
traj = mdtraj.load("/tmp/explicit_simulationSimulation/simulation/889_openff200_tip3p_0_0_output.h5")

In [ ]:
# centering the solute
traj.image_molecules(inplace=True)

In [ ]:
# saving topology for vmd
traj[0].save_pdb("/tmp/explicit_simulationSimulation/simulation/889_openff200_tip3p_0_0_output.pdb")
# saving trajectory for vmd
traj[::100].save_xtc("/tmp/explicit_simulationSimulation/simulation/889_openff200_tip3p_0_0_output.xtc")